# 機械学習 課題5

※ セルを増やしても構いませんが、すでに書かれているセルの内容を変更しないで下さい。

## 1. 特異値分解

特異値分解（singular value decomposition）について調べて説明せよ。さらに、主成分分析と特異値分解との違いや関連性について説明せよ。<!-- 1 -->

## 2. 特異値分解と最小二乗法

単回帰や重回帰の講義で例として用いた計画行列$\pmb{X}$と目的変数ベクトル$\pmb{y}$を考える。
$$
\begin{align}
\pmb{X} = \begin{pmatrix}
1 & 1 \\
1 & 3 \\
1 & 6 \\
1 & 8
\end{pmatrix},
\pmb{y} = \begin{pmatrix}
3 \\ 6 \\ 5 \\ 7
\end{pmatrix}
\end{align}
$$

(1) [numpy.linalg.svd](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)などの関数を用い、計画行列$\pmb{X}$の特異値分解、

\begin{align}
\pmb{X} = \pmb{U} \pmb{\Sigma} \pmb{V}^\top
\end{align}

を求め、$\pmb{U}$, $\pmb{\Sigma}$, $\pmb{V}$を用いて$\pmb{X}$を再構成できることを表示・確認せよ（求めた$\pmb{U}$, $\pmb{\Sigma}$, $\pmb{V}$を表示する必要はない）。<!-- 2(1) -->

In [23]:
import numpy as np

X = np.array([[1, 1],[1, 3],[1, 6],[1, 8]])
y = np.array([[3],[6],[5],[7]])

U, s, V = np.linalg.svd(X, full_matrices=True)

U_column_size ,V_row_size = U.shape[1], V.shape[0]
smat = np.zeros((U_column_size, V_row_size), dtype=np.float64)

s_size = len(s)
smat[:s_size ,:s_size] = np.diag(s)

U @ smat @ V.T

array([[1., 1.],
       [1., 3.],
       [1., 6.],
       [1., 8.]])

In [24]:
np.allclose(X, U @ smat @ V.T)

True

(2) 特異値分解に基づいて、[ムーア・ペンローズの擬似逆行列（一般化逆行列）](https://ja.wikipedia.org/wiki/%E6%93%AC%E4%BC%BC%E9%80%86%E8%A1%8C%E5%88%97)を求める計算式と、その導出の概略を説明せよ。<!-- 2(2) -->

行列$\pmb{A}$の擬似逆行列$\pmb{A}^{+}$は$\pmb{A} = \pmb{U} \pmb{\Sigma} \pmb{V}^\top$を用いて
$$\pmb{A}^{+} = \pmb{V} \pmb{\Sigma}^{+} \pmb{U}^\top$$
で求められる。

これは、$\pmb{V}, \pmb{U}$は直行行列であり、$\pmb{U}^\top = \pmb{U}^{-1}, \pmb{V}^\top = \pmb{V}^{-1}$が成立する.

$A^{+}$とは、$A A^{+} A = A$が成立するので、上記のようにおくと$\pmb{A} \pmb{A}^{+} \pmb{A} = \pmb{U} \pmb{\Sigma} \pmb{V}^{\top} \pmb{V}  \pmb{\Sigma}^{+} \pmb{U}^{\top} \pmb{A} = \pmb{U} \pmb{\Sigma} \pmb{E}  \pmb{\Sigma}^{+} \pmb{U}^{\top} \pmb{A} = \pmb{U} \pmb{U}^{\top} \pmb{A} = \pmb{E} \pmb{A} = \pmb{A}$が成立するので、擬似逆行列が以上ように表せることがわかる。

(3) (1)の結果と(2)の計算式を用い、計画行列$\pmb{X}$の疑似逆行列を求め、その結果から

\begin{align}
\lVert \pmb{X}\pmb{w} - \pmb{y} \rVert^2
\end{align}

を最小にする$\pmb{w}$、すなわち回帰係数を求め、その値を示せ（単回帰の時に求めた値と同じになることを確認せよ）。<!-- 2(3) -->

In [35]:
import numpy as np
from sklearn.linear_model import LinearRegression

pseudo_inverse_by_np = np.linalg.pinv(X)
smat_inverse = np.zeros((V_row_size, U_column_size), dtype=np.float64)
s_size = len(s)
smat_inverse[:s_size ,:s_size] = np.diag(1/s)
pseudo_inverse = V @ smat_inverse @ U.T

np.allclose(pseudo_inverse_by_np, pseudo_inverse) # True : np.linalg.pinv(X) == V @ smat_inverse @ U.T

True

In [34]:
regression = LinearRegression()
regression.fit(X, y)
regression.coef_

array([[0.        , 0.43103448]])

In [36]:
w = pseudo_inverse @ y
w

array([[3.31034483],
       [0.43103448]])

一致していない...